In [88]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from textblob import TextBlob
import re
from datetime import datetime

In [116]:
def get_author_and_date(url_series):
    result = []
    for url in url_series:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        article_date = soup.select('div[data-testid="article-body-timestamp"] time')[0].contents[0]
        article_author = soup.select('span.byline-name')[0].text
        result.append((article_author, article_date))
    return result

In [117]:
def get_NBC(limit=None):
    url = 'https://www.nbcnews.com/Israel-Hamas-war-Gaza-Strip-conflict'
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_NBC = pd.DataFrame(columns=['headline', 'description', 'url'])
        article_wrappers = soup.find_all('div', class_='wide-tease-item__info-wrapper')
        headlines_NBC = [article.find('h2', class_='wide-tease-item__headline') for article in article_wrappers]
        df_NBC['headline'] = [headline_NBC.text for headline_NBC in headlines_NBC]
        df_NBC['description'] = [article.find('div', class_='wide-tease-item__description').text for article in article_wrappers]
        df_NBC['url'] = [headline.parent['href'] for headline in headlines_NBC]
        authors_and_dates = get_author_and_date(df_NBC['url'])
        df_NBC['author'] = [author_and_date[0] for author_and_date in authors_and_dates]
        df_NBC['date'] = [author_and_date[1] for author_and_date in authors_and_dates]
        
        if limit:
            df_NBC = df_NBC.head(limit)
        return df_NBC
    else:
        print("Error: Response code", response.status_code)

In [71]:
results = get_NBC()
print(results)

                                             headline  \
0   About 100 people detained from Northeastern Un...   
1   Mass graves found at Gaza hospitals raided by ...   
2   Hamas is reviewing an Israeli proposal for a c...   
3   Saved from her dead mother's womb, baby Sabree...   
4   Columbia University says it has banned protest...   
5   As USC cancels commencement, Columbia students...   
6   Columbia protest at a stalemate as students re...   
7   Israel bombed areas it told Palestinians were ...   
8   U.S. reviewing whether to restrict military ai...   
9   Students resume pro-Palestinian protests at a ...   
10  Campus protests: Pro-Palestinian demonstration...   

                                          description  \
0   Northeastern said antisemitic slurs were heard...   
1   Reports from the hospital raised “renewed conc...   
2   There has been growing international pressure ...   
3   Doctors somehow managed to revive Sabreen — a ...   
4   Khymani James, a junior at

In [118]:
get_NBC()

,headline,description,url,author,date
0,Hamas releases video of Israeli hostages Keith...,"In the video, Siegel and Miran spoke of not be...",https://www.nbcnews.com/news/world/hamas-relea...,Mirna Alsharif,"April 27, 2024, 9:48 PM UTC"
1,About 100 people detained from Northeastern Un...,Northeastern said antisemitic slurs were heard...,https://www.nbcnews.com/news/us-news/people-de...,Mirna Alsharif,"April 27, 2024, 6:04 PM UTC"
2,Mass graves found at Gaza hospitals raided by ...,Reports from the hospital raised “renewed conc...,https://www.nbcnews.com/news/world/mass-graves...,Chantal Da Silva,"April 27, 2024, 5:00 PM UTC"
3,Hamas is reviewing an Israeli proposal for a c...,There has been growing international pressure ...,https://www.nbcnews.com/news/world/hamas-revie...,The Associated Press,"April 27, 2024, 4:01 PM UTC"
4,"Saved from her dead mother's womb, baby Sabree...",Doctors somehow managed to revive Sabreen — a ...,https://www.nbcnews.com/news/world/saved-dead-...,Alexander Smith,"April 27, 2024, 10:56 AM UTC"
5,Columbia University says it has banned protest...,"Khymani James, a junior at Columbia and a self...",https://www.nbcnews.com/news/us-news/columbia-...,Rebecca Cohen,"April 27, 2024, 4:35 AM UTC"
6,"As USC cancels commencement, Columbia students...",Their high school years were marred by the pan...,https://www.nbcnews.com/news/us-news/usc-cance...,Daniella Silva,"April 26, 2024, 12:00 AM UTC"
7,Columbia protest at a stalemate as students re...,Pro-Palestinian encampments stretch from New Y...,https://www.nbcnews.com/news/us-news/columbia-...,Madison Lambert,"April 26, 2024, 8:04 PM UTC"
8,Israel bombed areas it told Palestinians were ...,NBC News found Palestinians were killed in sev...,https://www.nbcnews.com/news/world/palestinian...,Hala Gorani,"April 26, 2024, 8:20 PM UTC"
9,U.S. reviewing whether to restrict military ai...,Israeli officials have been pressing Washingto...,https://www.nbcnews.com/news/world/us-sanction...,Abigail Williams,"April 26, 2024, 5:46 PM UTC"


In [152]:
def get_author_and_date(url_series):
    result = []
    for url in url_series:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        article_date = soup.select('div[data-testid="article-body-timestamp"] time')[0].contents[0]
        article_author = soup.select('span.byline-name')[0].text
        result.append((article_author, article_date))
    return result

In [161]:
# Function to perform sentiment analysis on text
def analyze_sentiment(text):
    if text:
        # Create a TextBlob object
        blob = TextBlob(text)
       
        sentiment = blob.sentiment
       
        return sentiment.polarity
    else:
        return None
# Function to fetch data from NBC News and perform sentiment analysis
def get_NBC_sentiment(url, limit=None):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_NBC = pd.DataFrame(columns=['headline', 'description', 'url', 'combined_text', 'sentiment', 'author', 'date'])
        article_wrappers = soup.find_all('div', class_='wide-tease-item__info-wrapper')
        headlines_NBC = [article.find('h2', class_='wide-tease-item__headline').text for article in article_wrappers]
        descriptions_NBC = [article.find('div', class_='wide-tease-item__description').text for article in article_wrappers]
        combined_texts = [headline + " " + description for headline, description in zip(headlines_NBC, descriptions_NBC)]
        urls_NBC = [article.find('a')['href'] for article in article_wrappers]
        
       
        
        #Analyzing the sentiment for the text
        sentiments = [analyze_sentiment(combined_text) for combined_text in combined_texts]

      
        # Create DataFrame
        df_NBC['headline'] = headlines_NBC
        df_NBC['description'] = descriptions_NBC
        df_NBC['url'] = urls_NBC
        df_NBC['combined_text'] = combined_texts
        df_NBC['sentiment'] = sentiments
        # df_NBC['author'] = authors
        # df_NBC['date'] = dates

        if limit:
            df_NBC = df_NBC.head(limit)
       
        return df_NBC
    else:
        print("Error: Response code", response.status_code)
        return None

# Example usage
url = 'https://www.nbcnews.com/Israel-Hamas-war-Gaza-Strip-conflict'
df_NBC_sentiment_Israel = get_NBC_sentiment(url, limit=5)

df_NBC_sentiment_Israel

,headline,description,url,combined_text,sentiment,author,date
0,Hamas releases video of Israeli hostages Keith...,"In the video, Siegel and Miran spoke of not be...",https://www.nbcnews.com/Israel-Hamas-war-Gaza-...,Hamas releases video of Israeli hostages Keith...,0.400,NaN,NaN
1,About 100 people detained from Northeastern Un...,Northeastern said antisemitic slurs were heard...,https://www.nbcnews.com/Israel-Hamas-war-Gaza-...,About 100 people detained from Northeastern Un...,0.000,NaN,NaN
2,Mass graves found at Gaza hospitals raided by ...,Reports from the hospital raised “renewed conc...,https://www.nbcnews.com/Israel-Hamas-war-Gaza-...,Mass graves found at Gaza hospitals raided by ...,0.000,NaN,NaN
3,Hamas is reviewing an Israeli proposal for a c...,There has been growing international pressure ...,https://www.nbcnews.com/Israel-Hamas-war-Gaza-...,Hamas is reviewing an Israeli proposal for a c...,0.000,NaN,NaN
4,"Saved from her dead mother's womb, baby Sabree...",Doctors somehow managed to revive Sabreen — a ...,https://www.nbcnews.com/Israel-Hamas-war-Gaza-...,"Saved from her dead mother's womb, baby Sabree...",-0.425,NaN,NaN


In [121]:
#Ukraine
def get_NBC_Ukraine(limit=None):
    url = 'https://www.nbcnews.com/world/russia-ukraine-news'
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_NBC_Ukraine = pd.DataFrame(columns=['headline', 'description', 'url'])
        article_wrappers = soup.find_all('div', class_='wide-tease-item__info-wrapper')
        headlines_NBC = [article.find('h2', class_='wide-tease-item__headline') for article in article_wrappers]
        df_NBC_Ukraine['headline'] = [headline_NBC.text for headline_NBC in headlines_NBC]
        df_NBC_Ukraine['description'] = [article.find('div', class_='wide-tease-item__description').text for article in article_wrappers]
        df_NBC_Ukraine['url'] = [headline.parent['href'] for headline in headlines_NBC]
        authors_and_dates = get_author_and_date(df_NBC_Ukraine['url'])
        df_NBC_Ukraine['author'] = [author_and_date[0] for author_and_date in authors_and_dates]
        df_NBC_Ukraine['date'] = [author_and_date[1] for author_and_date in authors_and_dates]
        
        if limit:
            df_NBC_Ukraine = df_NBC_Ukraine.head(limit)
        print(df_NBC_Ukraine)
        return df_NBC_Ukraine
        
    else:
        print("Error: Response code", response.status_code)
    
results = get_NBC_Ukraine()

                                             headline  \
0   Biden signs Ukraine aid, Israel funding and Ti...   
1   New U.S. military aid is finally heading to Uk...   
2   Russia detains ally of Defense Minister Shoigu...   
3   Mitch McConnell says Tucker Carlson and Trump'...   
4   They were promised jobs in Russia. They ended ...   
5   Rep. Marjorie Taylor Greene says Speaker Mike ...   
6   Zelenskyy says Ukraine has 'a chance at victor...   
7   Ukrainian and Western leaders laud U.S. aid pa...   
8   House passes critical aid to Ukraine, Israel a...   
9   Congress passes bill that could unlock billion...   
10  House advances Ukraine and Israel aid with rar...   

                                          description  \
0   The Senate on Tuesday passed the package of fo...   
1   For many in Kyiv, the news that the aid packag...   
2   Deputy Defense Minister Timur Ivanov was accus...   
3   The Senate Republican leader pointed to Carlso...   
4   Frustrated families of Ind

In [104]:
get_NBC_Ukraine()

                                             headline  \
0   Biden signs Ukraine aid, Israel funding and Ti...   
1   New U.S. military aid is finally heading to Uk...   
2   Russia detains ally of Defense Minister Shoigu...   
3   Mitch McConnell says Tucker Carlson and Trump'...   
4   They were promised jobs in Russia. They ended ...   
5   Rep. Marjorie Taylor Greene says Speaker Mike ...   
6   Zelenskyy says Ukraine has 'a chance at victor...   
7   Ukrainian and Western leaders laud U.S. aid pa...   
8   House passes critical aid to Ukraine, Israel a...   
9   Congress passes bill that could unlock billion...   
10  House advances Ukraine and Israel aid with rar...   

                                          description  \
0   The Senate on Tuesday passed the package of fo...   
1   For many in Kyiv, the news that the aid packag...   
2   Deputy Defense Minister Timur Ivanov was accus...   
3   The Senate Republican leader pointed to Carlso...   
4   Frustrated families of Ind

,headline,description,url,author,date
0,"Biden signs Ukraine aid, Israel funding and Ti...",The Senate on Tuesday passed the package of fo...,https://www.nbcnews.com/politics/congress/sena...,Sahil Kapur,"April 23, 2024, 11:00 AM UTC"
1,New U.S. military aid is finally heading to Uk...,"For many in Kyiv, the news that the aid packag...",https://www.nbcnews.com/news/world/us-military...,Daryna Mayer,"April 24, 2024, 11:15 AM UTC"
2,Russia detains ally of Defense Minister Shoigu...,Deputy Defense Minister Timur Ivanov was accus...,https://www.nbcnews.com/news/world/russia-ukra...,Reuters,"April 24, 2024, 12:31 PM UTC"
3,Mitch McConnell says Tucker Carlson and Trump'...,The Senate Republican leader pointed to Carlso...,https://www.nbcnews.com/politics/congress/mcco...,Sahil Kapur,"April 23, 2024, 8:33 PM UTC"
4,They were promised jobs in Russia. They ended ...,Frustrated families of Indian migrant workers ...,https://www.nbcnews.com/news/world/russia-ukra...,Muheet Ul Islam,"April 22, 2024, 9:19 AM UTC"
5,Rep. Marjorie Taylor Greene says Speaker Mike ...,Greene indicated she’ll move forward with trig...,https://www.nbcnews.com/politics/congress/rep-...,Summer Concepcion,"April 21, 2024, 5:57 PM UTC"
6,Zelenskyy says Ukraine has 'a chance at victor...,Ukraine's president thanked U.S. leaders for p...,https://www.nbcnews.com/politics/congress/ukra...,Alexandra Marquez,"April 21, 2024, 1:07 PM UTC"
7,Ukrainian and Western leaders laud U.S. aid pa...,The House swiftly approved $95 billion in fore...,https://www.nbcnews.com/news/world/ukrainian-w...,The Associated Press,"April 21, 2024, 10:32 AM UTC"
8,"House passes critical aid to Ukraine, Israel a...",The bills amount to a $95 billion aid package ...,https://www.nbcnews.com/politics/congress/hous...,Sahil Kapur,"April 20, 2024, 11:00 AM UTC"
9,Congress passes bill that could unlock billion...,More than $6 billion of the $300 billion in fr...,https://www.nbcnews.com/news/world/house-vote-...,Anna Schecter,"April 20, 2024, 3:15 PM UTC"


In [122]:
 #Performing the sentiment analysis
def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [130]:
def get_author_and_date(url_series):
    result = []
    for url in url_series:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        article_date = soup.select('div[data-testid="article-body-timestamp"] time')[0].contents[0]
        article_author = soup.select('span.byline-name')[0].text
        result.append((article_author, article_date))
    return result

In [167]:
def get_NBC_Ukraine_sentiment_score(url, limit=None):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_NBC_Ukraine = pd.DataFrame(columns=['headline', 'description', 'url', 'combined_text', 'authors', 'dates'])
        article_wrappers = soup.find_all('div', class_='wide-tease-item__info-wrapper')
        headlines_NBC = [article.find('h2', class_='wide-tease-item__headline') for article in article_wrappers]
        descriptions_NBC = [article.find('div', class_='wide-tease-item__description').text for article in article_wrappers]
        urls_NBC = [article.find('a')['href'] for article in article_wrappers]
        # authors_and_dates = get_author_and_date(df_NBC_Ukraine['url'])
        # df_NBC_Ukraine['author'] = [author_and_date[0] for author_and_date in authors_and_dates]
        # df_NBC_Ukraine['date'] = [author_and_date[1] for author_and_date in authors_and_dates]
        # urls_NBC = [headline.parent['href'] for headline in headlines_NBC]
        combined_texts = [headline.text + " " + description for headline, description in zip(headlines_NBC, descriptions_NBC)]
       
        # Analyze sentiment for each combined text
        sentiments = [analyze_sentiment(combined_text) for combined_text in combined_texts]

        # Create DataFrame
        df_NBC_Ukraine['headline'] = [headline.text for headline in headlines_NBC]
        df_NBC_Ukraine['description'] = descriptions_NBC
        # df_NBC_Ukraine['autho']
        df_NBC_Ukraine['url'] = urls_NBC
        df_NBC_Ukraine['combined_text'] = combined_texts
        df_NBC_Ukraine['headline_sentiment'] = sentiments
       
        if limit:
            df_NBC_Ukraine = df_NBC_Ukraine.head(limit)
       
        return df_NBC_Ukraine
    else:
        print("Error: Response code", response.status_code)
        return None

# Example usage
url = 'https://www.nbcnews.com/world/russia-ukraine-news'
df_NBC_Ukraine = get_NBC_Ukraine_sentiment_score(url, limit=10)

df_NBC_Ukraine

,headline,description,url,combined_text,authors,dates,headline_sentiment
0,"Biden signs Ukraine aid, Israel funding and Ti...",The Senate on Tuesday passed the package of fo...,https://www.nbcnews.com/world/russia-ukraine-news,"Biden signs Ukraine aid, Israel funding and Ti...",NaN,NaN,-0.125000
1,New U.S. military aid is finally heading to Uk...,"For many in Kyiv, the news that the aid packag...",https://www.nbcnews.com/world/russia-ukraine-news,New U.S. military aid is finally heading to Uk...,NaN,NaN,-0.055682
2,Russia detains ally of Defense Minister Shoigu...,Deputy Defense Minister Timur Ivanov was accus...,https://www.nbcnews.com/world/russia-ukraine-news,Russia detains ally of Defense Minister Shoigu...,NaN,NaN,0.214286
3,Mitch McConnell says Tucker Carlson and Trump'...,The Senate Republican leader pointed to Carlso...,https://www.nbcnews.com/world/russia-ukraine-news,Mitch McConnell says Tucker Carlson and Trump'...,NaN,NaN,-0.016667
4,They were promised jobs in Russia. They ended ...,Frustrated families of Indian migrant workers ...,https://www.nbcnews.com/world/russia-ukraine-news,They were promised jobs in Russia. They ended ...,NaN,NaN,-0.060000
5,Rep. Marjorie Taylor Greene says Speaker Mike ...,Greene indicated she’ll move forward with trig...,https://www.nbcnews.com/world/russia-ukraine-news,Rep. Marjorie Taylor Greene says Speaker Mike ...,NaN,NaN,0.000000
6,Zelenskyy says Ukraine has 'a chance at victor...,Ukraine's president thanked U.S. leaders for p...,https://www.nbcnews.com/world/russia-ukraine-news,Zelenskyy says Ukraine has 'a chance at victor...,NaN,NaN,0.168182
7,Ukrainian and Western leaders laud U.S. aid pa...,The House swiftly approved $95 billion in fore...,https://www.nbcnews.com/world/russia-ukraine-news,Ukrainian and Western leaders laud U.S. aid pa...,NaN,NaN,0.010000
8,"House passes critical aid to Ukraine, Israel a...",The bills amount to a $95 billion aid package ...,https://www.nbcnews.com/world/russia-ukraine-news,"House passes critical aid to Ukraine, Israel a...",NaN,NaN,0.000000
9,Congress passes bill that could unlock billion...,More than $6 billion of the $300 billion in fr...,https://www.nbcnews.com/world/russia-ukraine-news,Congress passes bill that could unlock billion...,NaN,NaN,0.166667


In [100]:
BASE_URL = 'https://www.nytimes.com'
def get_date_NYT(url_series):
    result = []
    for url in url_series:
        page = requests.get(BASE_URL + url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # Extracting article date
        article_date= soup.find('time')['datetime']
        result.append(article_date)
    return result

In [101]:
def extract_time(input_string):
    input_string = input_string.split('\n', 1)[-1].strip()
   
    pattern = r'(\d{1,2}:\d{2} (?:AM|PM)) EDT, (\w{3}) (\w+) (\d{1,2}), (\d{4})'

    match = re.match(pattern, input_string)
    if match:
        time_str, day_of_week, month_str, day_str, year_str = match.groups()
        month_num = datetime.strptime(month_str, '%B').month
        date_time = f"{month_num}/{day_str}/{year_str} {time_str}"

        return date_time

In [357]:
df_NYT_IsraelPalestine['date']

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: date, dtype: object

In [102]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def analyze_sentiment(text):
    if text:
        # Creating a textblob 
        blob = TextBlob(text)
       
        # Performing sentiment analysis
        sentiment = blob.sentiment
       
        
        return sentiment.polarity
    else:
        return None

def get_NYT_IsraelPalestine_sentiment(limit=None):
    url = 'https://www.nytimes.com/news-event/israel-hamas-gaza'
    driver = webdriver.Chrome()
    driver.get(url)
    df_NYT_IsraelPalestine = pd.DataFrame(columns=['headline', 'description', 'url', 'date', 'author', 'sentiment'])
    article_wrappers = driver.find_elements(By.CSS_SELECTOR, 'article.css-1l4spti')
    headlines_NYTIsraelPalestine = [article.find_element(By.CSS_SELECTOR, 'h3.css-1kv6qi').text.strip() for article in article_wrappers]
    descriptions_NYTIsraelPalestine = [article.find_element(By.CSS_SELECTOR, 'p.css-1pga48a').text.strip() for article in article_wrappers]
    urls_NYTIsraelPalestine = [article.find_element(By.CSS_SELECTOR, 'a.css-8hzhxf').get_attribute('href') for article in article_wrappers]
    authors_NYTIsraelPalestine = [article.find_element(By.CSS_SELECTOR, 'span.css-1n7hynb').text.strip() for article in article_wrappers]
    dates_NYTIsraelPalestine = [article.find_element(By.XPATH ,'..').find_element(By.CSS_SELECTOR, 'span[data-testid="todays-date"]').text for article in article_wrappers]
    combined_texts = [headline + " " + description for headline, description in zip(headlines_NYTIsraelPalestine, descriptions_NYTIsraelPalestine)]
    
    # Analyze sentiment for each combined text
    sentiments = [analyze_sentiment(combined_text) for combined_text in combined_texts]
    # Analyzing for every description
    # sentiments = [analyze_sentiment(description) for description in descriptions_NYTIsraelPalestine]

    # Create DataFrame
    df_NYT_IsraelPalestine['headline'] = headlines_NYTIsraelPalestine
    df_NYT_IsraelPalestine['description'] = descriptions_NYTIsraelPalestine
    df_NYT_IsraelPalestine['url'] = urls_NYTIsraelPalestine
    df_NYT_IsraelPalestine['author'] = authors_NYTIsraelPalestine
    df_NYT_IsraelPalestine['date'] = dates_NYTIsraelPalestine
    df_NYT_IsraelPalestine['sentiment'] = sentiments
    
    if limit:
        df_NYT_IsraelPalestine = df_NYT_IsraelPalestine.head(limit)
        return df_NYT_IsraelPalestine
    else:
        print("Error: Response code")
        return None

# Example usage
df_NYT_IsraelPalestine = get_NYT_IsraelPalestine_sentiment(limit=10)
df_NYT_IsraelPalestine

,headline,description,url,date,author,sentiment
0,Crackdowns at Three College Protests Lead to N...,The police made arrests at Northeastern Univer...,https://www.nytimes.com/2024/04/27/us/northeas...,"April 27, 2024","ANNA BETTS, MATTHEW EADIE and NICHOLAS BOGEL-B...",0.300000
1,Yair Lapid Says the World Misunderstands Israel,"Frustrated at the growing protest movement, th...",https://www.nytimes.com/2024/04/27/magazine/ya...,"April 27, 2024",LULU GARCIA-NAVARRO,-0.700000
2,Barnard Ends Suspensions for Most Student Prot...,The students had been among more than 100 who ...,https://www.nytimes.com/2024/04/26/nyregion/ba...,"April 26, 2024",CLAIRE FAHY,0.500000
3,Columbia’s University Senate Calls for an Inve...,The senators voted for a resolution that accus...,https://www.nytimes.com/2024/04/26/us/columbia...,"April 26, 2024",STEPHANIE SAUL and ANNA BETTS,0.000000
4,Pentagon Announces Additional $6 Billion in Mi...,The funds will allow Kyiv to purchase weapons ...,https://www.nytimes.com/2024/04/26/us/politics...,"April 26, 2024",JOHN ISMAY,0.000000
5,"For Fox News and Conservative Media, Student P...","On Fox and in other conservative outlets, the ...",https://www.nytimes.com/2024/04/26/business/fo...,"April 26, 2024",SANTUL NERKAR,0.128788
6,Why Gaza Protests on U.S. College Campuses Hav...,Experts say the partisan political context in ...,https://www.nytimes.com/2024/04/26/world/europ...,"April 26, 2024",AMANDA TAUB,-0.025000
7,Columbia Bars Student Protester Who Said ‘Zion...,"After video surfaced on social media, the stud...",https://www.nytimes.com/2024/04/26/nyregion/co...,"April 26, 2024",KATHERINE ROSMAN,-0.110101
8,Schumer Says Foreign Aid Victory Shows Congres...,The majority leader says the measure to help U...,https://www.nytimes.com/2024/04/26/us/politics...,"April 26, 2024",CARL HULSE,-0.058333
9,Army Begins Building Floating Aid Pier Off Gaz...,The structure is meant to allow humanitarian a...,https://www.nytimes.com/2024/04/26/us/politics...,"April 25, 2024",HELENE COOPER,0.000000


In [81]:
#NYT Ukraine_Russia
def get_NYT_UkraineRussia(limit=None):
    url = 'https://www.nytimes.com/news-event/ukraine-russia'
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_NYT_Ukraine_Russia = pd.DataFrame(columns=['headline', 'description', 'url', 'author'])
        article_wrappers = soup.find_all('article', class_='css-1l4spti')
        headlines_NYTUkraineRussia = [article.find('h3', class_='css-1kv6qi') for article in article_wrappers]
        df_NYT_Ukraine_Russia['headline'] = [headline_NYT.text if headline_NYT else None for headline_NYT in headlines_NYTUkraineRussia]
        df_NYT_Ukraine_Russia['description'] = [article.find('p', class_='css-1pga48a').text.strip() if article.find('p', class_='css-1pga48a') else None for article in article_wrappers]
        df_NYT_Ukraine_Russia['url'] = [headline.parent['href'] for headline in headlines_NYTUkraineRussia]
        df_NYT_Ukraine_Russia['author'] = soup.find_all('div', class_='css-1i4y2t3')
        df_NYT_Ukraine_Russia['author'] = [author.text.strip() if author else None for author in df_NYT_Ukraine_Russia['author']]
       
        
        if limit:
            df_NYT_Ukraine_Russia = df_NYT_Ukraine_Russia.head(limit)
        print(df_NYT_Ukraine_Russia)
        return df_NYT_Ukraine_Russia
        
    else:
        print("Error: Response code", response.status_code)
df_NYT_UkraineRussia = get_NYT_UkraineRussia(limit=5)
get_NYT_UkraineRussia()

                                            headline  \
0    Putin’s War Will Soon Reach Russians’ Tax Bills   
1  Macron, Battling the Far Right at Home, Pushes...   
2       These Issues Are Testing the U.S.-China Thaw   
3  NATO Puts on a Show of Force in the Shadow of ...   
4  Aid to Ukraine Is on the Way. Here’s How It Mi...   

                                         description  \
0  Russia’s president has signaled an increase in...   
1  In a major speech, France’s president returned...   
2  Secretary of State Antony J. Blinken is in Chi...   
3  The alliance’s largest exercises offer a previ...   
4  Weapons from the support package, considered “...   

                                                 url  \
0  /2024/04/27/world/europe/russia-taxes-ukraine-...   
1  /2024/04/25/world/europe/france-macron-far-rig...   
2  /2024/04/25/world/asia/blinken-china-united-st...   
3  /2024/04/24/us/politics/nato-exercise-russia-u...   
4   /2024/04/24/world/europe/ukraine-aid-senat

,headline,description,url,author
0,Putin’s War Will Soon Reach Russians’ Tax Bills,Russia’s president has signaled an increase in...,/2024/04/27/world/europe/russia-taxes-ukraine-...,By Paul Sonne
1,"Macron, Battling the Far Right at Home, Pushes...","In a major speech, France’s president returned...",/2024/04/25/world/europe/france-macron-far-rig...,By Roger Cohen and Aurelien Breeden
2,These Issues Are Testing the U.S.-China Thaw,Secretary of State Antony J. Blinken is in Chi...,/2024/04/25/world/asia/blinken-china-united-st...,"By Ana Swanson, David Pierson and Olivia Wang"
3,NATO Puts on a Show of Force in the Shadow of ...,The alliance’s largest exercises offer a previ...,/2024/04/24/us/politics/nato-exercise-russia-u...,By Helene Cooper
4,Aid to Ukraine Is on the Way. Here’s How It Mi...,"Weapons from the support package, considered “...",/2024/04/24/world/europe/ukraine-aid-senate.html,By Lara Jakes
5,‘Kharkiv Is Unbreakable’: A Battered City Carr...,For residents of Ukraine’s second-largest city...,/2024/04/24/world/europe/ukraine-kharkiv-russi...,By Marc Santora and Tyler Hicks
6,Here’s What’s in the Foreign Aid Package That ...,"Assistance for Ukraine, Israel and Taiwan is p...",/2024/04/23/us/politics/aid-bill-ukraine-israe...,By Robert Jimison
7,"In Ukraine, New American Technology Won the Da...",Project Maven was meant to revolutionize moder...,/2024/04/23/us/politics/ukraine-new-american-t...,By David E. Sanger
8,Ukraine War Helped Push World Military Spendin...,"The outlay reached $2.4 trillion last year, a ...",/2024/04/22/us/politics/ukraine-global-militar...,By Lara Jakes
9,Ukraine’s Race to Hold the Line,The days of lightning battlefield breakthrough...,/interactive/2024/04/22/world/europe/ukraine-w...,"By Josh Holder, Eric Schmitt and Thomas Gibbon..."


In [169]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def analyze_sentiment(text):
    if text:
        blob = TextBlob(text)
        sentiment = blob.sentiment
        return sentiment.polarity
    else:
        return None

def get_NYT_UkraineRussia(limit=None):
    url = 'https://www.nytimes.com/news-event/ukraine-russia'
    driver = webdriver.Chrome()
    driver.get(url)
    df_NYT_UkraineRussia = pd.DataFrame(columns=['headline', 'description', 'url', 'date', 'author'])
    article_wrappers = driver.find_elements(By.CSS_SELECTOR, 'article.css-1l4spti')
    headlines_NYT_UkraineRussia = [article.find_element(By.CSS_SELECTOR, 'h3.css-1kv6qi').text.strip() for article in article_wrappers]
    descriptions_NYT_UkraineRussia = [article.find_element(By.CSS_SELECTOR, 'p.css-1pga48a').text.strip() for article in article_wrappers]
    urls_NYT_UkraineRussia= [article.find_element(By.CSS_SELECTOR, 'a.css-8hzhxf').get_attribute('href') for article in article_wrappers]
    authors_NYT_UkraineRussia = [article.find_element(By.CSS_SELECTOR, 'span.css-1n7hynb').text.strip() for article in article_wrappers]
    dates_NYT_UkraineRussia = [article.find_element(By.XPATH ,'..').find_element(By.CSS_SELECTOR, 'span[data-testid="todays-date"]').text for article in article_wrappers]
    combined_texts = [headline + " " + description for headline, description in zip(headlines_NYT_UkraineRussia, descriptions_NYT_UkraineRussia)]

    sentiments = [analyze_sentiment(combined_text) for combined_text in combined_texts]
    
    df_NYT_UkraineRussia['headline'] = headlines_NYT_UkraineRussia
    df_NYT_UkraineRussia['description'] = descriptions_NYT_UkraineRussia
    df_NYT_UkraineRussia['url'] = urls_NYT_UkraineRussia
    df_NYT_UkraineRussia['author'] = authors_NYT_UkraineRussia
    df_NYT_UkraineRussia['date'] = dates_NYT_UkraineRussia
    df_NYT_UkraineRussia['headline_sentiment'] = sentiments
         
    if limit:
        df_NYT_UkraineRussia = df_NYT_UkraineRussia.head(limit)
        return df_NYT_UkraineRussia
    else:
        print("Error: Response code")

# Example usage
df_NYT_UkraineRussia = get_NYT_UkraineRussia(limit=10)
df_NYT_UkraineRussia


,headline,description,url,date,author,headline_sentiment
0,Putin’s War Will Soon Reach Russians’ Tax Bills,Russia’s president has signaled an increase in...,https://www.nytimes.com/2024/04/27/world/europ...,"April 27, 2024",PAUL SONNE,-0.066667
1,"Macron, Battling the Far Right at Home, Pushes...","In a major speech, France’s president returned...",https://www.nytimes.com/2024/04/25/world/europ...,"April 25, 2024",ROGER COHEN and AURELIEN BREEDEN,0.264643
2,These Issues Are Testing the U.S.-China Thaw,Secretary of State Antony J. Blinken is in Chi...,https://www.nytimes.com/2024/04/25/world/asia/...,"April 25, 2024","ANA SWANSON, DAVID PIERSON and OLIVIA WANG",0.000000
3,NATO Puts on a Show of Force in the Shadow of ...,The alliance’s largest exercises offer a previ...,https://www.nytimes.com/2024/04/24/us/politics...,"April 24, 2024",HELENE COOPER,0.400000
4,Aid to Ukraine Is on the Way. Here’s How It Mi...,"Weapons from the support package, considered “...",https://www.nytimes.com/2024/04/24/world/europ...,"April 24, 2024",LARA JAKES,-0.100000
5,‘Kharkiv Is Unbreakable’: A Battered City Carr...,For residents of Ukraine’s second-largest city...,https://www.nytimes.com/2024/04/24/world/europ...,"April 24, 2024",MARC SANTORA and TYLER HICKS,0.000000
6,Here’s What’s in the Foreign Aid Package That ...,"Assistance for Ukraine, Israel and Taiwan is p...",https://www.nytimes.com/2024/04/23/us/politics...,"April 23, 2024",ROBERT JIMISON,0.087500
7,"In Ukraine, New American Technology Won the Da...",Project Maven was meant to revolutionize moder...,https://www.nytimes.com/2024/04/23/us/politics...,"April 23, 2024",DAVID E. SANGER,-0.040909
8,Ukraine War Helped Push World Military Spendin...,"The outlay reached $2.4 trillion last year, a ...",https://www.nytimes.com/2024/04/22/us/politics...,"April 22, 2024",LARA JAKES,0.015000
9,Ukraine’s Race to Hold the Line,The days of lightning battlefield breakthrough...,https://www.nytimes.com/interactive/2024/04/22...,"April 22, 2024","JOSH HOLDER, ERIC SCHMITT and THOMAS GIBBONS-NEFF",-0.093750


In [266]:
df_NYT_UkraineRussia

,headline,description,url,author,sentiment
0,"Macron, Battling the Far Right at Home, Pushes...","In a major speech, France’s president returned...",/2024/04/25/world/europe/france-macron-far-rig...,By Roger Cohen and Aurelien Breeden,0.31250
1,These Issues Are Testing the U.S.-China Thaw,Secretary of State Antony J. Blinken is in Chi...,/2024/04/25/world/asia/blinken-china-united-st...,"By Ana Swanson, David Pierson and Olivia Wang",0.00000
2,Enduring Mayhem: Images From Year 3 of the War...,A photographic chronicle of the third year of ...,/2024/04/24/world/europe/images-ukraine-war-th...,,0.00000
3,NATO Puts on a Show of Force in the Shadow of ...,The alliance’s largest exercises offer a previ...,/2024/04/24/us/politics/nato-exercise-russia-u...,By Helene Cooper,0.40000
4,‘Kharkiv Is Unbreakable’: A Battered City Carr...,For residents of Ukraine’s second-largest city...,/2024/04/24/world/europe/ukraine-kharkiv-russi...,By Marc Santora and Tyler Hicks,0.00000
5,Here’s What’s in the Foreign Aid Package That ...,"Assistance for Ukraine, Israel and Taiwan is p...",/2024/04/23/us/politics/aid-bill-ukraine-israe...,By Robert Jimison,0.30000
6,"In Ukraine, New American Technology Won the Da...",Project Maven was meant to revolutionize moder...,/2024/04/23/us/politics/ukraine-new-american-t...,By David E. Sanger,-0.15000
7,Ukraine War Helped Push World Military Spendin...,"The outlay reached $2.4 trillion last year, a ...",/2024/04/22/us/politics/ukraine-global-militar...,By Lara Jakes,0.00000
8,Ukraine’s Race to Hold the Line,The days of lightning battlefield breakthrough...,/interactive/2024/04/22/world/europe/ukraine-w...,"By Josh Holder, Eric Schmitt and Thomas Gibbon...",-0.09375
9,Russian Attacks Crush Factories and Way of Lif...,The assaults have all but destroyed the factor...,/2024/04/21/world/europe/ukraine-industry-donb...,"By Andrew E. Kramer, Maria Varenikova and Nico...",0.20000


In [36]:
def get_CNN_Israel(limit=None):
    url = 'https://www.cnn.com/world/middleeast/israel'
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        df_CNN_Israel = pd.DataFrame(columns=['headline', 'description', 'urls', 'author_and_date']) 
        article_wrappers = soup.find_all('a', class_='container___link')
        df_CNN_Israel['headline'] = [article.find('div', class_='container__headline').text.strip() for article in article_wrappers]  
        df_CNN_Israel['urls'] = [link['href'] for link in soup.find_all('div', class_='zone__items')[0].find_all('a', href=True)]
        
        return df_CNN_Israel

    
    df_CNN_Israel()
    print(df_CNN_Israel)

In [37]:
BASE_URL = 'https://www.cnn.com'
url = f'{BASE_URL}/world/middleeast/israel'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
df_CNN_Israel = pd.DataFrame(columns=['headline', 'description', 'urls', 'author_and_date']) 
article_wrappers = soup.select('a.container__link--type-article span.container__headline-text')[1:]


In [38]:
def get_author_and_date_CNN(url_series):
    result = []
    for url in url_series:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        div_timestamp = soup.select('div.timestamp')[0]
        article_lead_author = soup.select('div.byline__names')[0]
        article_authors = [author.text for author in article_lead_author.select('span.byline__name')]
        article_authors.append(article_lead_author.contents[0].strip()[3:-1])
        article_authors = ', '.join(article_authors)
        result.append((article_authors, div_timestamp.text.strip()))
    return result

In [40]:
import re
from datetime import datetime


def extract_time(input_string):
    input_string = input_string.split('\n', 1)[-1].strip()
   
    pattern = r'(\d{1,2}:\d{2} (?:AM|PM)) EDT, (\w{3}) (\w+) (\d{1,2}), (\d{4})'

    match = re.match(pattern, input_string)
    if match:
        time_str, day_of_week, month_str, day_str, year_str = match.groups()
        month_num = datetime.strptime(month_str, '%B').month
        date_time = f"{month_num}/{day_str}/{year_str} {time_str}"

        return date_time


In [41]:
df_CNN_Israel['headline'] = [article.text.strip() for article in article_wrappers]  
df_CNN_Israel['url'] = [BASE_URL + article.parent.parent.parent['href'] for article in article_wrappers]
authors_and_dates = get_author_and_date_CNN(df_CNN_Israel['url'])
df_CNN_Israel['author'] = [author_and_date[0] for author_and_date in authors_and_dates]
df_CNN_Israel['date'] = [author_and_date[1] for author_and_date in authors_and_dates]
df_CNN_Israel['date'] = df_CNN_Israel['date'].apply(extract_time)
df_CNN_Israel['date'] = pd.to_datetime(df_CNN_Israel['date'], format='%m/%d/%Y %I:%M %p')

df_CNN_Israel

,headline,description,urls,author_and_date,url,author,date
0,Ship with 400 tons of food aid destined for Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/gaza...,"Zeena Saifi, Lauren Said-Moorhouse,",2024-04-27 13:03:00
1,Daughter of prominent Palestinian poet killed ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/shai...,"Zeena Saifi, Sana Noor Haq,",2024-04-27 11:12:00
2,Egyptian delegation arrives in Israel to reviv...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/middleeast/egyp...,"Jeremy Diamond, Eugenia Yosef, Alex Marquardt,...",2024-04-27 00:02:00
3,Blinken tells CNN the US has seen evidence of ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/blinke...,"Simone McCarthy,",2024-04-26 18:38:00
4,Hamas officials say group willing to disarm if...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/hama...,"Abbas Al Lawati, CN",2024-04-26 05:55:00
5,White House unbowed on support for Israel desp...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/joe-bi...,"Kayla Tausche, MJ Lee,",2024-04-26 05:00:00
6,Almost 400 bodies have been found in mass grav...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/gaza...,"Zeena Saifi, Abeer Salman, Ibrahim Dahman,",2024-04-25 23:53:00
7,‘I love you. Stay strong. Survive.’ Parents of...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/isra...,"Tara John,",2024-04-25 22:53:00
8,US military begins construction of pier off Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/politics/us-mil...,"Natasha Bertrand, Haley Britzky, Oren Lieberma...",2024-04-25 17:33:00
9,More than 300 bodies found in mass grave at Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/22/middleeast/khan...,"Abeer Salman, Ibrahim Dahman and Tim Lister, CN",2024-04-23 09:38:00


In [42]:
sentiments = df_CNN_Israel['headline'].apply(analyze_sentiment)
df_CNN_Israel['headline_sentiment'] = sentiments

df_CNN_Israel

,headline,description,urls,author_and_date,url,author,date,headline_sentiment
0,Ship with 400 tons of food aid destined for Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/gaza...,"Zeena Saifi, Lauren Said-Moorhouse,",2024-04-27 13:03:00,0.000000
1,Daughter of prominent Palestinian poet killed ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/shai...,"Zeena Saifi, Sana Noor Haq,",2024-04-27 11:12:00,0.150000
2,Egyptian delegation arrives in Israel to reviv...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/middleeast/egyp...,"Jeremy Diamond, Eugenia Yosef, Alex Marquardt,...",2024-04-27 00:02:00,0.000000
3,Blinken tells CNN the US has seen evidence of ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/blinke...,"Simone McCarthy,",2024-04-26 18:38:00,0.000000
4,Hamas officials say group willing to disarm if...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/hama...,"Abbas Al Lawati, CN",2024-04-26 05:55:00,0.250000
5,White House unbowed on support for Israel desp...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/joe-bi...,"Kayla Tausche, MJ Lee,",2024-04-26 05:00:00,0.000000
6,Almost 400 bodies have been found in mass grav...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/gaza...,"Zeena Saifi, Abeer Salman, Ibrahim Dahman,",2024-04-25 23:53:00,0.000000
7,‘I love you. Stay strong. Survive.’ Parents of...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/isra...,"Tara John,",2024-04-25 22:53:00,0.466667
8,US military begins construction of pier off Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/politics/us-mil...,"Natasha Bertrand, Haley Britzky, Oren Lieberma...",2024-04-25 17:33:00,-0.333333
9,More than 300 bodies found in mass grave at Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/22/middleeast/khan...,"Abeer Salman, Ibrahim Dahman and Tim Lister, CN",2024-04-23 09:38:00,0.500000


In [43]:
BASE_URL = 'https://www.cnn.com'
url = f'{BASE_URL}/world/europe/ukraine'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
df_CNN_Ukraine = pd.DataFrame(columns=['headline', 'description']) 
article_wrappers = soup.select('a.container__link--type-article span.container__headline-text')[1:]

In [44]:
def get_author_and_date_CNN(url_series):
    result = []
    for url in url_series:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        div_timestamp = soup.select('div.timestamp')[0]
        article_lead_author = soup.select('div.byline__names')[0]
        article_authors = [author.text for author in article_lead_author.select('span.byline__name')]
        article_authors.append(article_lead_author.contents[0].strip()[3:-1])
        article_authors = ', '.join(article_authors)
        result.append((article_authors, div_timestamp.text.strip()))
    return result

In [45]:
import re
from datetime import datetime


def extract_time(input_string):
    input_string = input_string.split('\n', 1)[-1].strip()
   
    pattern = r'(\d{1,2}:\d{2} (?:AM|PM)) EDT, (\w{3}) (\w+) (\d{1,2}), (\d{4})'

    match = re.match(pattern, input_string)
    if match:
        time_str, day_of_week, month_str, day_str, year_str = match.groups()
        month_num = datetime.strptime(month_str, '%B').month
        date_time = f"{month_num}/{day_str}/{year_str} {time_str}"

        return date_time


In [46]:
df_CNN_Ukraine['headline'] = [article.text.strip() for article in article_wrappers]  
df_CNN_Ukraine['url'] = [BASE_URL + article.parent.parent.parent['href'] for article in article_wrappers]
authors_and_dates = get_author_and_date_CNN(df_CNN_Ukraine['url'])
df_CNN_Ukraine['author'] = [author_and_date[0] for author_and_date in authors_and_dates]
df_CNN_Ukraine['date'] = [author_and_date[1] for author_and_date in authors_and_dates]
df_CNN_Ukraine['date'] = df_CNN_Ukraine['date'].apply(extract_time)
df_CNN_Ukraine['date'] = pd.to_datetime(df_CNN_Ukraine['date'], format='%m/%d/%Y %I:%M %p')

df_CNN_Ukraine

,headline,description,url,author,date
0,Russian oil refineries and military airfield t...,NaN,https://www.cnn.com/2024/04/27/europe/ukraine-...,"Lauren Said-Moorhouse, Olga Voitovych an",2024-04-27 09:37:00
1,US announces $6 billion long-term military aid...,NaN,https://www.cnn.com/2024/04/26/politics/us-lon...,"Haley Britzky,",2024-04-26 15:03:00
2,Russian deputy defense minister dismissed afte...,NaN,https://www.cnn.com/2024/04/24/europe/russian-...,"Jessie Gretener, Darya Tarasova, Christian Edw...",2024-04-25 08:29:00
3,US secretly sent long-range missiles to Ukrain...,NaN,https://www.cnn.com/2024/04/24/politics/us-sec...,"Oren Liebermann, Natasha Bertrand, Haley Britz...",2024-04-24 23:20:00
4,"As Ukraine waits for US military aid, Russia m...",NaN,https://www.cnn.com/2024/04/24/world/ukraine-a...,"Andrew Carey and Svitlana Vlasova, CN",2024-04-24 21:26:00
5,Ukraine tightens pressure on military age men ...,NaN,https://www.cnn.com/2024/04/23/europe/ukraine-...,"Radina Gigova,",2024-04-24 19:43:00
6,Biden signs foreign aid bill providing crucial...,NaN,https://www.cnn.com/2024/04/24/politics/biden-...,"Michael Williams, Arlette Saenz, Kevin Liptak,",2024-04-24 13:46:00
7,Right-wing media ruptures over Marjorie Taylor...,NaN,https://www.cnn.com/2024/04/26/media/right-win...,"Oliver Darcy, lysis b",2024-04-26 07:36:00
8,Ukraine’s AI-enabled drones are trying to disr...,NaN,https://www.cnn.com/2024/04/01/energy/ukrainia...,"Vasco Cotovio, Clare Sebastian, Allegra Goodwin,",2024-04-02 08:38:00
9,EU leaders endorse plan to tap frozen Russian ...,NaN,https://www.cnn.com/2024/03/21/business/frozen...,"Hanna Ziady, Mark Thompson,",2024-03-22 08:30:00


In [47]:
sentiments = df_CNN_Ukraine['headline'].apply(analyze_sentiment)
df_CNN_Ukraine['headline_sentiment'] = sentiments

df_CNN_Ukraine

,headline,description,url,author,date,headline_sentiment
0,Russian oil refineries and military airfield t...,NaN,https://www.cnn.com/2024/04/27/europe/ukraine-...,"Lauren Said-Moorhouse, Olga Voitovych an",2024-04-27 09:37:00,-0.050
1,US announces $6 billion long-term military aid...,NaN,https://www.cnn.com/2024/04/26/politics/us-lon...,"Haley Britzky,",2024-04-26 15:03:00,-0.100
2,Russian deputy defense minister dismissed afte...,NaN,https://www.cnn.com/2024/04/24/europe/russian-...,"Jessie Gretener, Darya Tarasova, Christian Edw...",2024-04-25 08:29:00,-0.025
3,US secretly sent long-range missiles to Ukrain...,NaN,https://www.cnn.com/2024/04/24/politics/us-sec...,"Oren Liebermann, Natasha Bertrand, Haley Britz...",2024-04-24 23:20:00,-0.400
4,"As Ukraine waits for US military aid, Russia m...",NaN,https://www.cnn.com/2024/04/24/world/ukraine-a...,"Andrew Carey and Svitlana Vlasova, CN",2024-04-24 21:26:00,-0.050
5,Ukraine tightens pressure on military age men ...,NaN,https://www.cnn.com/2024/04/23/europe/ukraine-...,"Radina Gigova,",2024-04-24 19:43:00,-0.100
6,Biden signs foreign aid bill providing crucial...,NaN,https://www.cnn.com/2024/04/24/politics/biden-...,"Michael Williams, Arlette Saenz, Kevin Liptak,",2024-04-24 13:46:00,-0.075
7,Right-wing media ruptures over Marjorie Taylor...,NaN,https://www.cnn.com/2024/04/26/media/right-win...,"Oliver Darcy, lysis b",2024-04-26 07:36:00,0.000
8,Ukraine’s AI-enabled drones are trying to disr...,NaN,https://www.cnn.com/2024/04/01/energy/ukrainia...,"Vasco Cotovio, Clare Sebastian, Allegra Goodwin,",2024-04-02 08:38:00,0.100
9,EU leaders endorse plan to tap frozen Russian ...,NaN,https://www.cnn.com/2024/03/21/business/frozen...,"Hanna Ziady, Mark Thompson,",2024-03-22 08:30:00,0.000


In [170]:
combined_df = pd.concat([df_CNN_Israel, df_CNN_Ukraine, df_NYT_IsraelPalestine, df_NYT_UkraineRussia, df_NBC_Ukraine, df_NBC_sentiment_Israel])

In [171]:
combined_df

,headline,description,urls,author_and_date,url,author,date,headline_sentiment,sentiment,combined_text,authors,dates
0,Ship with 400 tons of food aid destined for Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/gaza...,"Zeena Saifi, Lauren Said-Moorhouse,",2024-04-27 13:03:00,0.000000,NaN,NaN,NaN,NaN
1,Daughter of prominent Palestinian poet killed ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/27/middleeast/shai...,"Zeena Saifi, Sana Noor Haq,",2024-04-27 11:12:00,0.150000,NaN,NaN,NaN,NaN
2,Egyptian delegation arrives in Israel to reviv...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/middleeast/egyp...,"Jeremy Diamond, Eugenia Yosef, Alex Marquardt,...",2024-04-27 00:02:00,0.000000,NaN,NaN,NaN,NaN
3,Blinken tells CNN the US has seen evidence of ...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/blinke...,"Simone McCarthy,",2024-04-26 18:38:00,0.000000,NaN,NaN,NaN,NaN
4,Hamas officials say group willing to disarm if...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/hama...,"Abbas Al Lawati, CN",2024-04-26 05:55:00,0.250000,NaN,NaN,NaN,NaN
5,White House unbowed on support for Israel desp...,NaN,NaN,NaN,https://www.cnn.com/2024/04/26/politics/joe-bi...,"Kayla Tausche, MJ Lee,",2024-04-26 05:00:00,0.000000,NaN,NaN,NaN,NaN
6,Almost 400 bodies have been found in mass grav...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/gaza...,"Zeena Saifi, Abeer Salman, Ibrahim Dahman,",2024-04-25 23:53:00,0.000000,NaN,NaN,NaN,NaN
7,‘I love you. Stay strong. Survive.’ Parents of...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/middleeast/isra...,"Tara John,",2024-04-25 22:53:00,0.466667,NaN,NaN,NaN,NaN
8,US military begins construction of pier off Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/25/politics/us-mil...,"Natasha Bertrand, Haley Britzky, Oren Lieberma...",2024-04-25 17:33:00,-0.333333,NaN,NaN,NaN,NaN
9,More than 300 bodies found in mass grave at Ga...,NaN,NaN,NaN,https://www.cnn.com/2024/04/22/middleeast/khan...,"Abeer Salman, Ibrahim Dahman and Tim Lister, CN",2024-04-23 09:38:00,0.500000,NaN,NaN,NaN,NaN


In [ ]:
def create_data_table(headlines, descriptions,source):
    df = pd.DataFrame({
        'Headlines': headlines, 
        'Descriptions': descriptions,
        'Source': source
    })
    return df

In [379]:
data = {
    'headline'
    'description'
    'source'
    'author'

KeyError: ('headlines', 'descriptions', 'sentiment', 'newssource')

In [383]:
url = 'https://www.nytimes.com/news-event/israel-hamas-gaza'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')

In [386]:
soup.find('span', {"data-testid":"todays-date"})